In [8]:
# Standard libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Handeling .wav files

import librosa
from librosa import feature

# Machine Learning

from sklearn.pipeline import Pipeline

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, RepeatedStratifiedKFold

from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from functools import reduce

# data vizualisation

import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
def merge_pd(machine):
    
    # Opening the .csv files
    
    df_6dB=pd.read_csv(f'Librosa_features_{machine}_6dB.csv')
    df_0dB=pd.read_csv(f'Librosa_features_{machine}_0dB.csv')
    df_min6dB=pd.read_csv(f'Librosa_features_{machine}_-6dB.csv')
    
    # Dropping the 'Unnamed: 0' column
    
    df_6dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    df_0dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    df_min6dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    

    
    # Merging the .csv files into one DataFrame
    
    data_frames = [df_6dB, df_0dB, df_min6dB]
    df_merged = pd.concat(data_frames)
    
    
    return df_merged

In [10]:
valve = merge_pd('valve')

In [11]:
minority_class_len = len(valve[valve['normal(0)/abnormal(1)'] == 1])
majority_class_indices = valve[valve['normal(0)/abnormal(1)'] == 0].index
random_majority_indices = np.random.choice(majority_class_indices, minority_class_len , replace= False)
minority_class_indices  = valve[valve['normal(0)/abnormal(1)'] == 1].index

under_sample_indices = np.concatenate( [minority_class_indices , random_majority_indices])
under_sample = valve.loc[under_sample_indices]
X = under_sample.drop(columns = ['normal(0)/abnormal(1)'])
y = under_sample['normal(0)/abnormal(1)']

# 20% of the overal data will seperated for later validation of the model

X_model, X_valid, y_model, y_valid = train_test_split(X, y, test_size=0.2, random_state = 42, stratify = y)

# 60% of the overal data (75% of X_model, y_model) will be used to create a training set for the model
# 20% of the overal data (25% of X_model, y_model) will be used to create a testing set for the model

X_train, X_test, y_train, y_test = train_test_split(X_model,
                                                   y_model,
                                                   test_size = 0.25,
                                                   random_state = 10, stratify=y_model)

In [12]:
pipeline = Pipeline([('scaler', StandardScaler()), ('clf', SVC())])

In [13]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 56em; }</style>"))

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.feature_selection import RFE

y_pred = []
clfs = []

clfs.append(DecisionTreeClassifier())
clfs.append(RandomForestClassifier())
clfs.append(RandomForestClassifier(class_weight='balanced'))
#clfs.append(EasyEnsembleClassifier(importance_getter=index))

#feature_list = [6,13,21]

#for index in feature_list:
for index in range(4,22):
    
    for classifier in clfs:
        
        print("----------------------------------------------")
        print("----------------------------------------------")
        print(classifier)
        print("----------------------------------------------")
        print("----------------------------------------------")
        
        sel = RFE(classifier, n_features_to_select = index)
        sel.fit(X_train, y_train)
        features = X_train.columns[sel.get_support()]
        X_train_rfe = sel.transform(X_train)
        X_test_rfe = sel.transform(X_test)
        print('Selected Feature', index)
        print(features)
    
        classifier.fit(X_train_rfe, y_train)
        y_pred= classifier.predict(X_test_rfe)
        scores = cross_val_score(pipeline, X_train_rfe, y_train, cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
)


        print("----------------------------------------------")
        print("TRAIN-TEST")
        print("----------------------------------------------")


        print('confusion matrix', classifier)
        print(confusion_matrix(y_test, y_pred))
        print('classification report')
        print(classification_report(y_test, y_pred))
        print('accuracy score')
        print(accuracy_score(y_test, y_pred))
        
        X_valid_rfe = sel.transform(X_valid)
        y_pred = classifier.predict(X_valid_rfe)

        print("----------------------------------------------")
        print("TRAIN-VALIDATION")
        print("----------------------------------------------")

        print('confusion matrix', classifier)
        print(confusion_matrix(y_valid, y_pred))
        print('classification report')
        print(classification_report(y_valid, y_pred))
        print('accuracy score')
        print(accuracy_score(y_valid, y_pred))

----------------------------------------------
----------------------------------------------
DecisionTreeClassifier()
----------------------------------------------
----------------------------------------------
Selected Feature 4
Index(['melspectrogram_max', 'spectral_contrast', 'max perc', 'min perc'], dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[759 103]
 [ 32 831]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.88      0.92       862
           1       0.89      0.96      0.92       863

    accuracy                           0.92      1725
   macro avg       0.92      0.92      0.92      1725
weighted avg       0.92      0.92      0.92      1725

accuracy score
0.9217391304347826
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion mat

Selected Feature 6
Index(['melspectrogram_max', 'melspectrogram_corr', 'rms',
       'spectral_flatness ', 'max perc', 'min perc'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[765  97]
 [ 39 824]]
classification report
              precision    recall  f1-score   support

           0       0.95      0.89      0.92       862
           1       0.89      0.95      0.92       863

    accuracy                           0.92      1725
   macro avg       0.92      0.92      0.92      1725
weighted avg       0.92      0.92      0.92      1725

accuracy score
0.9211594202898551
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[763 100]
 [ 24 838]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.88      0

Selected Feature 8
Index(['melspectrogram_max', 'melspectrogram_corr', 'melspectrogram_std',
       'mfcc', 'spectral_contrast', 'spectral_flatness ', 'max perc',
       'min perc'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[745 117]
 [ 51 812]]
classification report
              precision    recall  f1-score   support

           0       0.94      0.86      0.90       862
           1       0.87      0.94      0.91       863

    accuracy                           0.90      1725
   macro avg       0.90      0.90      0.90      1725
weighted avg       0.90      0.90      0.90      1725

accuracy score
0.9026086956521739
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[763 100]
 [ 42 820]]
classification report
              precision    recall  f1-score 

Selected Feature 10
Index(['melspectrogram_max', 'melspectrogram_corr', 'melspectrogram_std',
       'mfcc', 'rms', 'spectral_contrast', 'spectral_flatness ', 'max perc',
       'min harm', 'min perc'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[739 123]
 [ 52 811]]
classification report
              precision    recall  f1-score   support

           0       0.93      0.86      0.89       862
           1       0.87      0.94      0.90       863

    accuracy                           0.90      1725
   macro avg       0.90      0.90      0.90      1725
weighted avg       0.90      0.90      0.90      1725

accuracy score
0.8985507246376812
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[749 114]
 [ 41 821]]
classification report
              precision 

Selected Feature 12
Index(['melspectrogram_max', 'melspectrogram_corr', 'melspectrogram_std',
       'mfcc', 'rms', 'spectral_contrast', 'spectral_flatness ',
       'zero_crossing_rate', 'max harm', 'max perc', 'min harm', 'min perc'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[743 119]
 [ 47 816]]
classification report
              precision    recall  f1-score   support

           0       0.94      0.86      0.90       862
           1       0.87      0.95      0.91       863

    accuracy                           0.90      1725
   macro avg       0.91      0.90      0.90      1725
weighted avg       0.91      0.90      0.90      1725

accuracy score
0.903768115942029
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[769  94]
 [ 37 825]]
classificatio

----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[804  58]
 [ 28 835]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.93      0.95       862
           1       0.94      0.97      0.95       863

    accuracy                           0.95      1725
   macro avg       0.95      0.95      0.95      1725
weighted avg       0.95      0.95      0.95      1725

accuracy score
0.9501449275362319
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[817  46]
 [ 29 833]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       863
           1       0.95      0.97      0.96       862

    accuracy                           

----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier()
[[801  61]
 [ 34 829]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.93      0.94       862
           1       0.93      0.96      0.95       863

    accuracy                           0.94      1725
   macro avg       0.95      0.94      0.94      1725
weighted avg       0.95      0.94      0.94      1725

accuracy score
0.9449275362318841
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix RandomForestClassifier()
[[814  49]
 [ 26 836]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.94      0.96       863
           1       0.94      0.97      0.96       862

    accuracy                           0.96      1725
   macro avg       0.96      0.

----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[740 122]
 [ 39 824]]
classification report
              precision    recall  f1-score   support

           0       0.95      0.86      0.90       862
           1       0.87      0.95      0.91       863

    accuracy                           0.91      1725
   macro avg       0.91      0.91      0.91      1725
weighted avg       0.91      0.91      0.91      1725

accuracy score
0.9066666666666666
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[752 111]
 [ 36 826]]
classification report
              precision    recall  f1-score   support

           0       0.95      0.87      0.91       863
           1       0.88      0.96      0.92       862

    accuracy                           0.91      1725
   macro avg       0.92      0.

Selected Feature 18
Index(['melspectrogram', 'melspectrogram_max', 'melspectrogram_sum',
       'melspectrogram_corr', 'melspectrogram_std', 'mfcc', 'rms',
       'spectral_centroid', 'spectral_bandwidth', 'spectral_contrast',
       'spectral_flatness ', 'spectral_rolloff', 'zero_crossing_rate',
       'mean perc', 'max harm', 'max perc', 'min harm', 'min perc'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[802  60]
 [ 37 826]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.93      0.94       862
           1       0.93      0.96      0.94       863

    accuracy                           0.94      1725
   macro avg       0.94      0.94      0.94      1725
weighted avg       0.94      0.94      0.94      1725

accuracy score
0.943768115942029
------------------------------------

Selected Feature 20
Index(['melspectrogram', 'melspectrogram_min', 'melspectrogram_max',
       'melspectrogram_sum', 'melspectrogram_corr', 'melspectrogram_std',
       'mfcc', 'rms', 'spectral_centroid', 'spectral_bandwidth',
       'spectral_contrast', 'spectral_flatness ', 'spectral_rolloff',
       'zero_crossing_rate', 'mean harm', 'mean perc', 'max harm', 'max perc',
       'min harm', 'min perc'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier()
[[788  74]
 [ 38 825]]
classification report
              precision    recall  f1-score   support

           0       0.95      0.91      0.93       862
           1       0.92      0.96      0.94       863

    accuracy                           0.94      1725
   macro avg       0.94      0.94      0.94      1725
weighted avg       0.94      0.94      0.94      1725

accuracy score
0.9350724637681159
----------------